In [1]:
!pip install tensorflow-gpu  # Install TensorFlow GPU version
!pip install transformers    # Install Hugging Face Transformers


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import json
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# JSON data for inference
json_data = '''
[
    {
        "url": "https://gossipchimp.com/bigbangs-taeyang-spills-on-adjusting-to-promoting-with-third-and-fourth-gen-k-pop-idols/",
        "articleId": "c8b8796f2f164ff3bbd82a94df5a0e70",
        "country": "us",
        "language": "en",
        "pubDate": "2023-04-30T23:38:28+00:00",
        "title": "BIGBANG’s Taeyang Spills On Adjusting To Promoting With Third And Fourth Gen K-Pop Idols",
        "summary": "BIGBANG's Taeyang has made his solo comeback for the first time in six years and recently released his second solo mini-album, Down To Earth. In a brand new interview with Vogue Singapore, TaeyANG opens up about adjusting to the traits of the third- and fourth-generation and the way it has affected his promotions. He also shared that he wished to embrace these adjustments and actively took half in latest promotional ways, corresponding to dance challenges with fellow idols.Taeyang also discussed the origin of his hit single, “Eyes, Nose, Lips.”",
        "description": "Taeyang is embracing the business adjustments.",
        "content": "BIGBANG‘s Taeyang made his solo comeback for the first time in six years this year and recently released his second solo mini-album, Down To Earth."
    }
]
'''

# Extract text content from JSON fields
data = json.loads(json_data)
text_fields = [data[0]['title'], data[0]['summary'], data[0]['description'], data[0]['content']]
text = ' '.join(text_fields)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenize the text
tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, padding='max_length',
                                       return_attention_mask=True, return_tensors='tf')

# Perform inference using GPU (if available)
with tf.device('/GPU:0'):
    outputs = model(tokenized_text['input_ids'], token_type_ids=None, attention_mask=tokenized_text['attention_mask'])

# Get the predicted sentiment
logits = outputs.logits.numpy()
predicted_class = tf.argmax(logits, axis=1).numpy()[0]

sentiment_labels = ['Negative', 'Neutral', 'Positive']  # Assuming 3 classes for sentiment analysis
predicted_sentiment = sentiment_labels[predicted_class]

print(f"The predicted sentiment is: {predicted_sentiment}")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The predicted sentiment is: Neutral
